In [1]:
!pip install ultralytics mlflow --quiet

In [2]:
from ultralytics import YOLO
import ultralytics
import torch
import mlflow
import os
import pandas as pd

print(f"Ultralytics YOLOv8 version: {ultralytics.__version__}")
print("CUDA available:", torch.cuda.is_available())

print(f"Ultralytics YOLOv8 version: {ultralytics.__version__}")

Ultralytics YOLOv8 version: 8.3.145
CUDA available: True
Ultralytics YOLOv8 version: 8.3.145


In [4]:
# Paths
data_yaml_path = "synthetic_data/data.yaml"  # your dataset YAML
run_name = "lp-detector"
output_dir = f"runs/detect/{run_name}"

# Hyperparameters
params = {
    "model": "yolov8n.pt",
    "data": data_yaml_path,
    "epochs": 50,
    "imgsz": 640,
    "batch": 16
}


In [ ]:
# Load and train model
model = YOLO("runs/detect/lp-detector15/weights/last.pt")
results = model.train(
    data=params["data"],
    epochs=params["epochs"],
    imgsz=params["imgsz"],
    batch=params["batch"],
    name=run_name,
    resume=True
)

print("✅ Training complete.")

Ultralytics 8.3.145  Python-3.12.3 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=synthetic_data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\detect\lp-detector15\weights\last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=lp-detector15, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

train: Scanning C:\ML\RandomTopMovie\LicensePlate\synthetic_data\train\labels.cache... 6501 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6501/6501 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 181.888.0 MB/s, size: 96.2 KB)


val: Scanning C:\ML\RandomTopMovie\LicensePlate\synthetic_data\testval\labels.cache... 2167 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2167/2167 [00:00<?, ?it/s]


Plotting labels to runs\detect\lp-detector15\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Resuming training runs\detect\lp-detector15\weights\last.pt from epoch 41 to 50 total epochs
MLflow: logging run_id(2bfa8886fb3b42a88b733bbdc64289dd) to runs\mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs\mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\lp-detector15
Starting training for 50 epochs...
Closing dataloader mosaic


In [ ]:
mlflow.set_experiment("YOLOv8-LicensePlate-Detection")

with mlflow.start_run(run_name=run_name):
    # Log training parameters
    mlflow.log_params(params)

    # Paths to results
    weights_dir = os.path.join(output_dir, "weights")
    best_model = os.path.join(weights_dir, "best.pt")
    last_model = os.path.join(weights_dir, "last.pt")
    results_csv = os.path.join(output_dir, "results.csv")

    # Log weights
    mlflow.log_artifact(best_model, artifact_path="models")
    mlflow.log_artifact(last_model, artifact_path="models")

    # Log metrics from results.csv
    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)
        last_row = df.iloc[-1].to_dict()
        mlflow.log_metrics({k: v for k, v in last_row.items() if isinstance(v, (int, float))})
        mlflow.log_artifact(results_csv, artifact_path="metrics")

    # Log output images
    for file in os.listdir(output_dir):
        if file.endswith((".png", ".jpg")):
            mlflow.log_artifact(os.path.join(output_dir, file), artifact_path="plots")

    print("✅ MLflow logging complete.")
